# PubTator Networks
In this notebook we'll explore the creation of networks using grounded PubTator entity annotations.

In [54]:
import pandas as pd
import networkx as nx
from itertools import combinations
from collections import defaultdict
import numpy as np

## Gene-only co-occurrence network

In [4]:
genes = pd.read_csv('../data/pubtator/gene_only_partial_20Nov2023.csv')
genes.head()

,paperId,ann_text,ann_type,db_grounding
0,7bff04897ec52618c4adacb0122cddc455b255e3,Aphelenchus avenae,Species,70226
1,a8713d21e63e63ae733f656067032413b39d41e3,Artemia franciscana,Species,6661
2,a8713d21e63e63ae733f656067032413b39d41e3,Artemia franciscana,Species,6661
3,c2a1a6318d1902752bd59724cb3b6d55eaa842bb,mammalian,Species,9606
4,415502b49e3e0e412408fdf9a4653754ffb68ece,Escherichia coli,Species,562


In [8]:
genes.shape[0], len(genes.db_grounding.unique())

(1284, 189)

In [5]:
genes.ann_type.unique()

array(['Species', 'Genus', 'Strain', 'CellLine'], dtype=object)

We requested Gene and only got these types, so something is definitely wrong; but I'm going to continue on with network building using these types.

In [12]:
list(combinations(genes.db_grounding.unique(), 2))[:5]

[(70226, 6661), (70226, 9606), (70226, 562), (70226, 947166), (70226, 470)]

In [66]:
genes_graph = nx.MultiGraph()
nodes = {db_ground: defaultdict(list) for db_ground in genes.db_grounding.unique()}
for i, row in genes.iterrows():
    nodes[row['db_grounding']]['ann_text'].append(row.ann_text)
    nodes[row['db_grounding']]['ann_type'].append(row.ann_type)
    nodes[row['db_grounding']]['paperId'].append(row.paperId)
for node, attrs in nodes.items():
    for name, attr in attrs.items():
        try:
            nodes[node][name] = ' | '.join(list(set(attr)))
        except TypeError:
            changed = [i if isinstance(i, str) else 'None' for i in list(set(attr))]
            nodes[node][name] = ' |'.join(changed)
nodes_to_add = [(n, attrs) for n, attrs in nodes.items()]
genes_graph.add_nodes_from(nodes_to_add)
edge_id = 0
for paper in genes.paperId.unique():
    to_join = genes[genes['paperId'] == paper]
    edges_to_add = combinations(to_join.db_grounding.unique(), 2)
    edges_with_ids = []
    for e in edges_to_add:
        edge = (e[0], e[1], edge_id)
        edge_id += 1
        edges_with_ids.append(edge)
    genes_graph.add_edges_from(edges_with_ids, label=paper)

In [67]:
nx.write_graphml(genes_graph, '../data/kg/genes_but_species_750_co_occurrence_20Nov2023.graphml')